<a href="https://colab.research.google.com/github/hhodek/Project4_CreditScoreClassification/blob/main/NNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neural Network Model - Original

In [ ]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder




#spark_version = 'spark-3.4.1'
#os.environ['SPARK_VERSION']=spark_version
#!apt-get update
#!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
#!tar xf $SPARK_VERSION-bin-hadoop3.tgz
#!pip install -q findspark

# Set Environment Variables
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
#import findspark
#findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#spark = SparkSession.Builder().master("local").appName("thisThing").getOrCreate()
traindf = pd.read_csv("/content/drive/MyDrive/Copy of Copy of train.csv")

traindf

,ID,CustomerID,Month,Name,Age,SSN,Occupation,AnnualIncome,MonthlyInhandSalary,NumBankAccounts,...,CreditMix,OutstandingDebt,CreditUtilizationRatio,CreditHistoryAge,PaymentofMinAmount,TotalEMIpermonth,Amountinvestedmonthly,PaymentBehaviour,MonthlyBalance,CreditScore
0,0x1602,CUS0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,NaN,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.415295,HighspentSmallvaluepayments,312.494089,Good
1,0x1603,CUS0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.280222,LowspentLargevaluepayments,284.629162,Good
2,0x1604,CUS0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521,LowspentMediumvaluepayments,331.209863,Good
3,0x1605,CUS0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.458074,LowspentSmallvaluepayments,223.451310,Good
4,0x1606,CUS0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153,HighspentMediumvaluepayments,341.489231,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,NaN,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.971333,HighspentLargevaluepayments,479.866228,Poor
99996,0x25fea,CUS0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,NaN,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.185950,HighspentMediumvaluepayments,496.651610,Poor
99997,0x25feb,CUS0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.028477,HighspentLargevaluepayments,516.809083,Poor
99998,0x25fec,CUS0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.672582,LowspentLargevaluepayments,319.164979,Standard


In [ ]:
columns_to_drop = ["ID", "CustomerID", "Month", "Name", "Age",
                   "SSN", "Occupation", "TypeofLoan", "InterestRate", "ChangedCreditLimit","MonthlyInhandSalary",
                   "PaymentofMinAmount", "TotalEMIpermonth", "Amountinvestedmonthly",
                   "PaymentBehaviour"]
traindf1 = traindf.drop(columns=columns_to_drop)

traindf1['CreditHistoryAge'] = traindf1['CreditHistoryAge'].str[:2]



In [ ]:
train_clean_df = traindf1.dropna(subset=['CreditMix'])
train_clean_df = train_clean_df.fillna(0)
train_clean_df['CreditHistoryAge'] = pd.to_numeric(train_clean_df['CreditHistoryAge'], errors='coerce')
bin_mapping = {
    'Good': 'Good/Standard',
    'Standard': 'Good/Standard',
    'Poor': 'Poor'
}

# Create a new column for the binned categories
train_clean_df['target'] = train_clean_df['CreditScore'].map(bin_mapping)
train_clean_df = train_clean_df.drop(columns = 'CreditScore')
train_clean_df

,AnnualIncome,NumBankAccounts,NumCreditCard,NumofLoan,Delayfromduedate,NumofDelayedPayment,NumCreditInquiries,CreditMix,OutstandingDebt,CreditUtilizationRatio,CreditHistoryAge,MonthlyBalance,target
1,19114.12,3,4,4,-1,0.0,4.0,Good,809.98,31.944960,0,284.629162,Good/Standard
2,19114.12,3,4,4,3,7.0,4.0,Good,809.98,28.609352,22,331.209863,Good/Standard
3,19114.12,3,4,4,5,4.0,4.0,Good,809.98,31.377862,22,223.451310,Good/Standard
4,19114.12,3,4,4,6,0.0,4.0,Good,809.98,24.797347,22,341.489231,Good/Standard
5,19114.12,3,4,4,8,4.0,4.0,Good,809.98,27.262259,22,340.479212,Good/Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99991,20002.88,10,8,5,33,25.0,9.0,Bad,3571.70,37.140784,6,337.362988,Good/Standard
99993,39628.99,4,6,2,23,0.0,3.0,Good,502.38,29.135447,31,400.104466,Good/Standard
99997,39628.99,4,6,2,27,6.0,3.0,Good,502.38,41.255522,31,516.809083,Poor
99998,39628.99,4,6,2,20,0.0,3.0,Good,502.38,33.638208,31,319.164979,Good/Standard


In [ ]:
#Binning CreditMix Column
# bin_map = {
   # 'Good': 'Good',
  #  'Standard': 'Good',
  #  'Bad': 'Bad'
#}

# Create a new column for the binned categories
#train_clean_df['CreditMixBin'] = train_clean_df['CreditMix'].map(bin_map)
#train_clean_df = train_clean_df.drop(columns = 'CreditMix')
#train_clean_df

In [ ]:
# Split target column from dataset
y = train_clean_df['target']
X = train_clean_df.drop(columns='target',axis=1)

In [ ]:
X = pd.get_dummies(X)
# Preview the data
X.head()

,AnnualIncome,NumBankAccounts,NumCreditCard,NumofLoan,Delayfromduedate,NumofDelayedPayment,NumCreditInquiries,OutstandingDebt,CreditUtilizationRatio,CreditHistoryAge,MonthlyBalance,CreditMix_Bad,CreditMix_Good,CreditMix_Standard
1,19114.12,3,4,4,-1,0.0,4.0,809.98,31.944960,0,284.629162,0,1,0
2,19114.12,3,4,4,3,7.0,4.0,809.98,28.609352,22,331.209863,0,1,0
3,19114.12,3,4,4,5,4.0,4.0,809.98,31.377862,22,223.451310,0,1,0
4,19114.12,3,4,4,6,0.0,4.0,809.98,24.797347,22,341.489231,0,1,0
5,19114.12,3,4,4,8,4.0,4.0,809.98,27.262259,22,340.479212,0,1,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [ ]:
nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=len(X_train.columns)))

# Hidden layer 1
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 80)                1200      
                                                                 
 dense_11 (Dense)            (None, 30)                2430      
                                                                 
 dense_12 (Dense)            (None, 1)                 31        
                                                                 
Total params: 3661 (14.30 KB)
Trainable params: 3661 (14.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
nn.fit(X_train_scaled, y_train_encoded, epochs=100)

Epoch 1/100
1871/1871 [==============================] - 4s 2ms/step - loss: 0.4523 - accuracy: 0.7847
Epoch 2/100
1871/1871 [==============================] - 3s 2ms/step - loss: 0.4400 - accuracy: 0.7936
Epoch 3/100
1871/1871 [==============================] - 4s 2ms/step - loss: 0.4367 - accuracy: 0.7952
Epoch 4/100
1871/1871 [==============================] - 5s 3ms/step - loss: 0.4352 - accuracy: 0.7968
Epoch 5/100
1871/1871 [==============================] - 3s 2ms/step - loss: 0.4341 - accuracy: 0.7968
Epoch 6/100
1871/1871 [==============================] - 3s 2ms/step - loss: 0.4333 - accuracy: 0.7974
Epoch 7/100
1871/1871 [==============================] - 4s 2ms/step - loss: 0.4326 - accuracy: 0.7979
Epoch 8/100
1871/1871 [==============================] - 7s 4ms/step - loss: 0.4319 - accuracy: 0.7986
Epoch 9/100
1871/1871 [==============================] - 4s 2ms/step - loss: 0.4315 - accuracy: 0.7992
Epoch 10/100
1871/1871 [==============================] - 4s 2ms/step - l

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

624/624 - 1s - loss: 0.4232 - accuracy: 0.8183 - 921ms/epoch - 1ms/step
Loss: 0.42324042320251465, Accuracy: 0.818263828754425


In [ ]:
# Assuming y_test_encoded and y_pred_encoded are your true labels and predicted labels
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test_encoded, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Calculate the confusion matrix, accuracy score, and classification report
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

y_pred_encoded = nn.predict(X_test_scaled)
y_pred_encoded = (y_pred_encoded > 0.5).astype(int)

conf_matrix = confusion_matrix(y_test_encoded, y_pred_encoded)
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
class_report = classification_report(y_test_encoded, y_pred_encoded)

print("Confusion Matrix:")
print(conf_matrix)

print("\nAccuracy Score:", accuracy)

print("\nClassification Report:")
print(class_report)





624/624 - 1s - loss: 0.4285 - accuracy: 0.8019 - 903ms/epoch - 1ms/step
Loss: 0.42845645546913147, Accuracy: 0.8018745183944702
624/624 [==============================] - 1s 1ms/step
Confusion Matrix:
[[12712  1436]
 [ 2517  3287]]

Accuracy Score: 0.801874498797113

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.90      0.87     14148
           1       0.70      0.57      0.62      5804

    accuracy                           0.80     19952
   macro avg       0.77      0.73      0.74     19952
weighted avg       0.79      0.80      0.80     19952



Optimization of Model
- These were run in another notebook so they were copied and pasted here to compile it all.
- The changes made for each optimization attempt is noted at the top and the results are noted at the bottom

In [ ]:
# How the Keras tuner was ran

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=14))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

    # Import the kerastuner library
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

  # Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train_encoded,epochs=20,validation_data=(X_test_scaled,y_test_encoded))



#Keras Tuner Results:


#Trial 56 Complete [00h 01m 14s]
#val_accuracy: 0.7932037115097046

#Best val_accuracy So Far: 0.7942060828208923
#Total elapsed time: 00h 54m 20s


# Get top model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(1)
for param in top_hyper:
    print(param.values)

  #{'activation': 'tanh', 'first_units': 5, 'num_layers': 6, 'units_0': 5, 'units_1': 7, 'units_2': 5,
  #'units_3': 1, 'units_4': 3, 'units_5': 3, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2,
  #'tuner/round': 2, 'tuner/trial_id': '0038'}

  # Evaluate the model using the test data - No drop monthlyinhandsal & 6 layers (x10 - 7 9 7 3 5 5 1) @ 20 epochs
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
#624/624 - 1s - loss: 0.4295 - accuracy: 0.8112 - 1s/epoch - 2ms/step
#Loss: 0.4294900894165039, Accuracy: 0.8111968636512756



In [ ]:
# Evaluate the model using the test data - No drop monthlyinhandsal
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

		#624/624 - 1s - loss: 0.4230 - accuracy: 0.8150 - 865ms/epoch - 1ms/step
		#Loss: 0.42297783493995667, Accuracy: 0.8150060176849365

In [ ]:
#BEST
#Kept the ChangedCreditLimit column

#624/624 - 2s - loss: 0.4232 - accuracy: 0.8183 - 2s/epoch - 3ms/step
#		Loss: 0.42324042320251465, Accuracy: 0.818263828754425
#		624/624 [==============================] - 3s 4ms/step
#		Confusion Matrix:
#		[[12805  1343]
#		 [ 2283  3521]]

#		Accuracy Score: 0.8182638331996792

#		Classification Report:
#		              precision    recall  f1-score   support

#		           0       0.85      0.91      0.88     14148
#		           1       0.72      0.61      0.66      5804

In [ ]:
# Evaluate the model using the test data - 2 hidden layers 80, 30
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

	#	624/624 - 2s - loss: 0.4256 - accuracy: 0.8103 - 2s/epoch - 4ms/step
	#	Loss: 0.42556577920913696, Accuracy: 0.8103448152542114

In [ ]:
# Evaluate the model using the test data - 2 relu layers and 5 bins (MIHS Col) and 2 layers 30 10
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
#624/624 - 1s - loss: 0.4294 - accuracy: 0.8091 - 1s/epoch - 2ms/step
#Loss: 0.4293609857559204, Accuracy: 0.8090918064117432

In [ ]:
# Evaluate the model using the test data - 2 relu layers and 3 bins (MIHS) and 2 layers 30 10
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
#624/624 - 1s - loss: 0.4294 - accuracy: 0.8091 - 1s/epoch - 2ms/step
#Loss: 0.4293609857559204, Accuracy: 0.8090918064117432

In [ ]:
# Evaluate the model using the test data - 3 hidden layers 80, 30, 10
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
#624/624 - 1s - loss: 0.4243 - accuracy: 0.8113 - 916ms/epoch - 1ms/step
#Loss: 0.4243035316467285, Accuracy: 0.8112971186637878